script to look at combingin spending into one consistent set of data

also try to help with auto tagging the spending 

---
So far in this script:

we have:
- read in spending data from first direct and amex
- added in tags from the tagged files taken from pythonista
- put in place a placeholder "untagged" if there is no tag there

In another notbook we can look at inferring tags from relatednes. 

In [5]:
ls

20161126_85579686.QIF    20190119_85579686.CSV    data-nov.txt
20170106_85579686.QIF    20190219_85579686.CSV    ofx-1.csv
20170504_85579686.QIF    20190312_85579686.CSV    ofx-2.csv
20170801_85579686.QIF    README.md                ofx-3.csv
20181220_85579686-1.CSV  data-dec.txt             spending_analysis.ipynb
20190109_85579686.CSV    data-jan-19.txt          test-repprt.txt


In [4]:
import csv
import glob
import collections
from difflib import SequenceMatcher

# Read in the data from local disk

In [5]:
def get_first_direct_csv_files():
    fd_csv_files = glob.glob("/Users/ianm/Dropbox/workbench/monthly-tracker/*85579686.CSV")
    return fd_csv_files 

In [6]:
def get_amex_csv_files():
    amex_csv_files = glob.glob("/Users/ianm/Dropbox/workbench/monthly-tracker/ofx*")
    return amex_csv_files

In [7]:
fd_csv_files = get_first_direct_csv_files()
amex_csv_files = get_amex_csv_files()

# Process CSV files from first direct

In [8]:
def reformat_fd_date(date):
    reformatted_date_parts = date.split("/")
    reformatted_date = reformatted_date_parts[-1] + "-" + reformatted_date_parts[1] + "-" + reformatted_date_parts[0]
#     print(reformatted_date)
    return reformatted_date

In [9]:
def read_fd_csv_file(filename):
    spends = []
    with open(filename) as csv_file:
        csv_reader = csv.reader(csv_file)
        next(csv_reader) # skip the header row! 
        for row in csv_reader:
            date = reformat_fd_date(row[0])
            description = row[1]
            amount = row[2]
            spends.append((date, description, amount))
    return spends

In [10]:
fd_spending = []
for file in fd_csv_files:
    fd_info = read_fd_csv_file(fd_csv_files[0])
    fd_spending.extend(fd_info)
#     for spend in fd_info:
#         if spend in fd_spending:
#             print(spend), 
#             print("is already added")
#         else:
#            fd_spending.append(spend)

In [11]:
de_duped_fd_spending = list(set(fd_spending))

In [12]:
de_duped_fd_spending.sort()
len(de_duped_fd_spending)

188

# Process files from american express

In [13]:
def read_amex_csv_file(filename):
    spends = []
    with open(filename) as csv_file:
        csv_reader = csv.reader(csv_file)
        next(csv_reader) # skip the header row! 
        for row in csv_reader:
            date = reformat_fd_date(row[0])
            description = row[3]
            amount = row[2]
            spends.append((date, description, amount))
    return spends

In [14]:
amex_spends = []
for file in amex_csv_files:
    amex_spend = read_amex_csv_file(file)
    amex_spends.extend(amex_spend)

In [15]:
de_duped_amexx_spending = list(set(amex_spends))

# merge amex and first direct speding.

In [16]:
all_spending = de_duped_fd_spending.copy()
all_spending.extend(de_duped_amexx_spending)
len(all_spending)

419

# get all tagged data that we stored in the other notebook

In [17]:
%store -r all_tagged_data

In [18]:
len(all_tagged_data)

425

# get similarity between all tagged data and already loaded spending data

In [32]:
def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

In [19]:
matched_count = 0 
for spend in all_tagged_data:
    spend_test_string = spend[0] + spend[1] + spend[2]
    for existing_spend in all_spending:
        check_string = existing_spend[0] + existing_spend[1] + existing_spend[2]
        similar_metric = similar(spend_test_string, check_string)
        if similar_metric > 0.933: # visual inspection tells us these are all matches
            tag = spend[3]
print(matched_count)

0


In [20]:
matched_count = 0 
all_spend_with_tags = []
for existing_spend in all_spending:
    spend_test_string = existing_spend[0] + existing_spend[1] + existing_spend[2]
    spend_with_tags = [existing_spend[0], existing_spend[1], existing_spend[2], "untagged"]
    tag_match = False    
    for spend in all_tagged_data:
        check_string = spend[0] + spend[1] + spend[2]
        similar_metric = similar(spend_test_string, check_string)
        if similar_metric > 0.933: # visual inspection tells us these are all matches
            tag = spend[3]
            spend_with_tags = [existing_spend[0], existing_spend[1], existing_spend[2], tag]
            all_spend_with_tags.append(spend_with_tags)
            tag_match = True
            break
    if not tag_match: all_spend_with_tags.append(spend_with_tags)

In [21]:
len(all_spend_with_tags)

419

In [19]:
%store all_spend_with_tags

Stored 'all_spend_with_tags' (list)


In [23]:
%store -r all_spend_with_tags

In [24]:
all_spend_with_tags[0:3]

[['2018-12-18', '404758 85579694    INTERNAL TRANSFER', '130.00', 'untagged'],
 ['2018-12-18', 'AMERICAN EXPRESS   WWW.AMERICANE', '-393.17', 'untagged'],
 ['2018-12-18', 'CO-OP GROUP FOOD   108942', '-3.15', 'food']]

# try to do some auto-tagging guess work on the now clean tagged data

In [20]:
tagged_list = []
untagged_list = []
for spend in all_spend_with_tags:
    if spend[3] == 'untagged':
        untagged_list.append(spend)
    else:
        tagged_list.append(spend)

In [21]:
%store tagged_list

Stored 'tagged_list' (list)


In [22]:
%store untagged_list

Stored 'untagged_list' (list)


In [57]:
#TODO have a think about this function, as I've created a number of bugs in it, and it's a little bit subtle
def create_groups(spending_list):
    processed = []
    spending_groups = []
    compare_list = spending_list.copy()
    for spend in spending_list:
        grouping = []
        if spend not in processed: grouping.append(spend) # this was it, this was where the problem was! 
        processed.append(spend) # this is our memory
        compare_list.remove(spend)
        for compare in compare_list: # look at all of the other items in the comparison group 
            similar_metric = similar(spend[1], compare[1])
            if similar_metric > 0.6:
                if compare not in processed:
                    grouping.append(compare)
                    processed.append(compare)  
        if grouping: spending_groups.append(grouping) # only add groups that are non-empty
    return spending_groups

In [49]:
untagged_groups = create_groups(untagged_list)
tagged_groups = create_groups(tagged_list)

In [119]:
total = 0
for group in untagged_groups:
    group_items = len(group)
    total += group_items
total, len(untagged_groups)

(183, 183)

In [50]:
len(untagged_groups)

71

In [54]:
total = 0
for group in tagged_groups:
    group_items = len(group)
    total += group_items
total, len(tagged_list), len(tagged_groups)

(236, 236, 72)

In [116]:
len(untagged_list)

183

In [35]:
len(all_spend_with_tags), len(untagged_list), len(tagged_list)

(419, 183, 236)

In [36]:
%store tagged_groups

Stored 'tagged_groups' (list)


In [37]:
%store untagged_groups

Stored 'untagged_groups' (list)


# let's try to infer tags based on old tag groups

In [38]:
%store -r

In [55]:
untagged_groups[-30:]

[[['2019-02-23', 'MOSLER A C G       FLIGHT', '600.00', 'untagged']],
 [['2019-02-25', 'BREAD AND BUTTER   LONDON', '-35.35', 'untagged'],
  ['2018-10-19', 'BROADWAY HAIR LONDON', ' 20.00', 'untagged']],
 [['2019-02-25', 'WWW.RMG.CO.UK      WWW', '-21.35', 'untagged']],
 [['2019-02-27',
   'ADVICE CONFIRMS    RBS27029HE5A9S74   ALFRED P SLOAN FDN',
   '313.40',
   'untagged']],
 [['2019-03-09',
   'CASH SAINSBY MAR09 112-118 King@19:32',
   '-160.00',
   'untagged']],
 [['2018-10-29', 'PAYMENT RECEIVED - THANK YOU', ' -190.70', 'untagged'],
  ['2018-10-16', 'PAYMENT RECEIVED - THANK YOU', ' -232.68', 'untagged'],
  ['2019-01-10', 'PAYMENT RECEIVED - THANK YOU', ' -100.00', 'untagged'],
  ['2019-01-12', 'PAYMENT RECEIVED - THANK YOU', ' -15.00', 'untagged'],
  ['2018-12-18', 'PAYMENT RECEIVED - THANK YOU', ' -119.00', 'untagged'],
  ['2019-01-08', 'PAYMENT RECEIVED - THANK YOU', ' -40.00', 'untagged'],
  ['2019-01-18', 'PAYMENT RECEIVED - THANK YOU', ' -65.00', 'untagged'],
  ['2019-01-

In [40]:
len(tagged_list)

236

In [58]:
from collections import defaultdict

In [67]:
tag_suggestions = defaultdict(set) # use set so that we only generate unique items in the suggestions. 
for spend in untagged_list:
    for tagged_spend in tagged_list:
        similar_metric = similar(spend[1], tagged_spend[1])
        if similar_metric > 0.6:
            tag_suggestions[tuple(spend)].add(tagged_spend[3])

# We can now generate suggestions for groups of untagged items 

In [71]:
for untagged_group in untagged_groups:
    for spend in untagged_group:
        print(spend, tag_suggestions[tuple(spend)])
    print("")

['2018-12-18', '404758 85579694    INTERNAL TRANSFER', '130.00', 'untagged'] set()
['2018-12-22', '404758 85579694    INTERNAL TRANSFER', '4500.00', 'untagged'] set()
['2019-01-21', '404772 12471736    INTERNAL TRANSFER', '624.00', 'untagged'] set()
['2019-02-19', '404758 85579694    INTERNAL TRANSFER', '300.00', 'untagged'] set()
['2019-02-22', '404758 85579694    INTERNAL TRANSFER', '160.00', 'untagged'] set()

['2018-12-18', 'AMERICAN EXPRESS   WWW.AMERICANE', '-393.17', 'untagged'] set()
['2019-01-04', 'AMERICAN EXPRESS   WWW.AMERICANE', '-350.00', 'untagged'] set()
['2019-01-07', 'AMERICAN EXPRESS   WWW.AMERICANE', '-300.00', 'untagged'] set()
['2019-01-15', 'AMERICAN EXPRESS   WWW.AMERICANE', '-500.00', 'untagged'] set()
['2019-01-15', 'AMERICAN EXPRESS   WWW.AMERICANE', '-900.00', 'untagged'] set()
['2019-01-16', 'AMERICAN EXPRESS   WWW.AMERICANE', '-500.00', 'untagged'] set()
['2019-01-24', 'AMERICAN EXPRESS   WWW.AMERICANE', '-279.06', 'untagged'] set()
['2019-02-12', 'AMERICA